In [1]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [2]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)
    skip = getattr(module, "skip", False)

    if skip:
        print("  Marked for skip")
        continue

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: anfext
  First time seen. Benchmarking.


benchmark: 100%|â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆ| 10000/10000 [00:08<00:00, 1118.61it/s]
c:\Users\shich\Src\bottleneck_transmission\src\benchmark_utils.py:385: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_full.groupby("bin")


{'basic': {'expected_sample_burst_size': 3.013934706974564,
           'expected_sample_data_size': 16.49346528942008,
           'expected_time_to_recover': 99.13940000000001,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.16138891298515018,
             'bit_redundancy': 415.69700000000006,
             'ideal_bit_efficency': 0.19088032898727186,
             'ideal_bit_redundancy': 339.11076130497713,
             'ideal_packet_efficiency': 0.9544016449363593,
             'ideal_packet_redundancy': 67.82215226099542,
             'ideal_time_to_recover': 83.82215226099542,
             'ideal_transmitted_bits': 419.11076130497713,
             'packet_efficiency': 0.806944564925751,
             'packet_redundancy': 83.13940000000001,
             'permeability': 0.8454978773423625,
             'saturation': 16.0,
             'transmitted_bits': 495.69700000000006}}
Found implementation: anfext2
  First time seen. Benchmarking.


benchmark: 100%|â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆ| 10000/10000 [00:04<00:00, 2072.79it/s]


{'basic': {'expected_sample_burst_size': 2.9927802153968885,
           'expected_sample_data_size': 16.51069006781013,
           'expected_time_to_recover': 39.64249999999999,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.4036072397048623,
             'bit_redundancy': 118.21249999999995,
             'ideal_bit_efficency': 0.47676637217108453,
             'ideal_bit_redundancy': 87.7970693186652,
             'ideal_packet_efficiency': 2.3838318608554223,
             'ideal_packet_redundancy': 17.55941386373304,
             'ideal_time_to_recover': 33.55941386373304,
             'ideal_transmitted_bits': 167.7970693186652,
             'packet_efficiency': 2.0180361985243116,
             'packet_redundancy': 23.64249999999999,
             'permeability': 0.8465513997283988,
             'saturation': 16.0,
             'transmitted_bits': 198.21249999999995}}
Found implementation: anfext2l
  First time seen. Benchmarking.


...

{'basic': {'expected_sample_burst_size': 2.996312854311656,
           'expected_sample_data_size': 16.509793980734663,
           'expected_time_to_recover': 32.1452,
           'packet_size': 5,
           'payload_bits': 25.0},
 'derived': {'bit_efficency': 0.1555442181103244,
             'bit_redundancy': 135.726,
             'ideal_bit_efficency': 0.18377347043665065,
             'ideal_bit_redundancy': 111.03704572046954,
             'ideal_packet_efficiency': 0.9188673521832532,
             'ideal_packet_redundancy': 22.20740914409391,
             'ideal_time_to_recover': 27.20740914409391,
             'ideal_transmitted_bits': 136.03704572046954,
             'packet_efficiency': 0.777721090551622,
             'packet_redundancy': 27.145200000000003,
             'permeability': 0.8463910364251555,
             'saturation': 5.0,
             'transmitted_bits': 160.726}}
Found implementation: single_binary_fullrank
  First time seen. Benchmarking.


benchmark: 100%|â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆ| 10000/10000 [00:02<00:00, 3465.92it/s]


{'basic': {'expected_sample_burst_size': 2.9916201117318435,
           'expected_sample_data_size': 16.694493216280925,
           'expected_time_to_recover': 13.3218,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.3491030390603996,
             'bit_redundancy': 43.35559567122584,
             'ideal_bit_efficency': 0.4116616120694495,
             'ideal_bit_redundancy': 33.233291654069106,
             'ideal_packet_efficiency': 2.0583080603472474,
             'ideal_packet_redundancy': 6.646658330813821,
             'ideal_time_to_recover': 11.297339196568652,
             'ideal_transmitted_bits': 56.48669598284326,
             'packet_efficiency': 1.7455151953019976,
             'packet_redundancy': 8.671119134245169,
             'permeability': 0.8480339891432579,
             'saturation': 4.650680865754831,
             'transmitted_bits': 66.609}}
Found implementation: single_binary_fullrank2
  First time see

benchmark: 100%|â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆ| 10000/10000 [00:02<00:00, 3503.24it/s]


{'basic': {'expected_sample_burst_size': 2.9875659085543402,
           'expected_sample_data_size': 16.49390099944912,
           'expected_time_to_recover': 13.8503,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.3357819589290362,
             'bit_redundancy': 45.998095671225855,
             'ideal_bit_efficency': 0.3966026667311193,
             'ideal_bit_redundancy': 35.378083251562835,
             'ideal_packet_efficiency': 1.983013333655597,
             'ideal_packet_redundancy': 7.075616650312565,
             'ideal_time_to_recover': 11.726297516067396,
             'ideal_transmitted_bits': 58.63148758033699,
             'packet_efficiency': 1.678909794645181,
             'packet_redundancy': 9.19961913424517,
             'permeability': 0.8466457416855516,
             'saturation': 4.650680865754831,
             'transmitted_bits': 69.25150000000001}}


### LT vs ANFEXT2L benchmark sweep
This section benchmarks LT and `anfext2l_undegenerate` across multiple payload sizes, caches the distributions, and plots the expected time-to-recover (TTR) and bit efficiency per payload.

In [4]:
from __future__ import annotations

import importlib
import json
from datetime import datetime
from pathlib import Path
import math

import pandas as pd
import plotly.graph_objects as go
from IPython.display import display
from tqdm.auto import tqdm

import impls.anfext2l_undegenerate as anfext2l_undegenerate
import impls.lt as lt
from benchmark_utils import benchmark, compute_distribution_stats

# Always reload to pick up local code edits when rerunning the cell
importlib.reload(anfext2l_undegenerate)
importlib.reload(lt)

# -----------------------------------------------------------------------------
# Configuration (adjust N or payload grids if you want to re-run with new params)
# -----------------------------------------------------------------------------
N = 5


def _payload_sizes_for_anfext2l(
    n: int, min_bits: int, max_bits: int, max_payload: int | None = None
) -> list[int]:
    base = (1 << n) - 2
    sizes: list[int] = []
    m = 1
    while True:
        payload = base**m
        bits = math.log2(payload)
        if bits > max_bits or (max_payload is not None and payload > max_payload):
            break
        if bits >= min_bits:
            sizes.append(payload)
        m += 1
    return sizes


ANF_OVERRIDE_FN = getattr(anfext2l_undegenerate, "override_D", None)
ANF_MAX_PAYLOAD = ANF_OVERRIDE_FN(N) if ANF_OVERRIDE_FN is not None else None
LT_OVERRIDE_FN = getattr(lt, "override_D", None)

PAYLOAD_BITS_BY_SCHEME = {
    "lt": [1 << bits for bits in (8, 9, 10, 11, 12, 13, 14, 15, 16)],
    "anfext2l_undegenerate": _payload_sizes_for_anfext2l(
        N,
        min_bits=8,
        max_bits=65,
        max_payload=ANF_MAX_PAYLOAD,
    ),
}
BENCHMARK_CFG = dict(passes=200, iters_bound=400, processes=None, progress_update=100)
RESULTS_PATH = Path("research/lt_vs_anfext2l_results.jsonl")
RESULTS_PATH.parent.mkdir(parents=True, exist_ok=True)

SCHEMES = {
    "lt": {
        "producer": lt.producer_constructor,
        "recoverer": lt.recoverer_constructor,
        "override": LT_OVERRIDE_FN,
    },
    "anfext2l_undegenerate": {
        "producer": anfext2l_undegenerate.producer_constructor,
        "recoverer": anfext2l_undegenerate.recoverer_constructor,
        "override": ANF_OVERRIDE_FN,
    },
}


def _load_records(path: Path) -> list[dict]:
    if not path.exists():
        return []
    records: list[dict] = []
    with path.open("r", encoding="utf-8") as fh:
        for line in fh:
            line = line.strip()
            if line:
                records.append(json.loads(line))
    return records


def _append_record(path: Path, record: dict) -> None:
    with path.open("a", encoding="utf-8") as fh:
        fh.write(json.dumps(record) + "\n")


records = _load_records(RESULTS_PATH)
existing_keys = {(rec["scheme"], rec["D"], rec["N"]) for rec in records}
plan = [
    (scheme, payload_size)
    for scheme, payloads in PAYLOAD_BITS_BY_SCHEME.items()
    for payload_size in payloads
]
pending = [
    (scheme, payload_size)
    for scheme, payload_size in plan
    if (scheme, payload_size, N) not in existing_keys
]

if pending:
    total_jobs = len(pending)
    print(f"Running {total_jobs} new benchmarks (N={N}).")
    with tqdm(total=total_jobs, desc=f"Benchmarks (N={N})") as bar:
        for scheme, payload_size in pending:
            impl = SCHEMES[scheme]
            D = payload_size
            payload_bits = math.log2(D)
            override = impl.get("override")
            if override is not None:
                max_d = override(N)
                if D > max_d:
                    raise ValueError(
                        f"Payload size D={D} (bits={payload_bits:.2f}) exceeds {scheme} support for N={N}"
                    )
            bench_result = benchmark(
                impl["producer"],
                impl["recoverer"],
                N,
                D,
                **BENCHMARK_CFG,
            )
            record = {
                "scheme": scheme,
                "payload_bits": payload_bits,
                "N": N,
                "D": D,
                "timestamp": datetime.utcnow().isoformat(),
                "benchmark_cfg": BENCHMARK_CFG,
                "result": bench_result,
            }
            records.append(record)
            _append_record(RESULTS_PATH, record)
            bar.update(1)
else:
    print(f"All requested payload benchmarks already cached for N={N}.")

records_df = pd.DataFrame(records)
records_df = records_df[records_df["N"] == N].copy()
if records_df.empty:
    raise RuntimeError(
        "No cached benchmark rows for the current N; run the cell again to generate them."
    )

summary_rows = []
for rec in records_df.to_dict("records"):
    stats = compute_distribution_stats(rec["N"], rec["D"], rec["result"])
    metrics = stats["metrics"]
    summary_rows.append(
        {
            "scheme": rec["scheme"],
            "payload_bits": rec["payload_bits"],
            "expected_ttr": metrics["basic"]["expected_time_to_recover"],
            "bit_efficiency": metrics["derived"]["bit_efficency"],
        }
    )

summary_df = pd.DataFrame(summary_rows).sort_values(["scheme", "payload_bits"])
print("Cached benchmark summary:")
display(summary_df)

fig_ttr = go.Figure()
for scheme, group in summary_df.groupby("scheme"):
    group = group.sort_values("payload_bits")
    fig_ttr.add_trace(
        go.Scatter(
            x=group["payload_bits"],
            y=group["expected_ttr"],
            mode="lines+markers",
            name=f"{scheme} E[TTR]",
            line=dict(shape="spline"),
            hovertemplate="scheme=%{text}<br>payload=%{x} bits<br>E[TTR]=%{y:.2f} packets<extra></extra>",
            text=[scheme] * len(group),
        )
    )

fig_ttr.update_layout(
    title=f"Expected time-to-recover vs payload size (N={N})",
    xaxis_title="payload bits",
    yaxis_title="expected TTR (packets)",
    template="plotly_white",
)
fig_ttr.show()

fig_eff = go.Figure()
for scheme, group in summary_df.groupby("scheme"):
    group = group.sort_values("payload_bits")
    fig_eff.add_trace(
        go.Scatter(
            x=group["payload_bits"],
            y=group["bit_efficiency"],
            mode="lines+markers",
            name=f"{scheme} bit efficiency",
            line=dict(shape="spline"),
            hovertemplate="scheme=%{text}<br>payload=%{x} bits<br>efficiency=%{y:.4f}<extra></extra>",
            text=[scheme] * len(group),
        )
    )

fig_eff.update_layout(
    title=f"Bit efficiency vs payload size (N={N})",
    xaxis_title="payload bits",
    yaxis_title="payload bits per transmitted bit",
    template="plotly_white",
)
fig_eff.show()

Running 21 new benchmarks (N=5).


Benchmarks (N=5): 100%|██████████| 21/21 [00:38<00:00,  1.85s/it]

Cached benchmark summary:


,scheme,payload_bits,expected_ttr,bit_efficiency
9,anfext2l_undegenerate,9.813781,11.690,0.167900
10,anfext2l_undegenerate,14.720672,12.410,0.237239
11,anfext2l_undegenerate,19.627562,13.145,0.298632
12,anfext2l_undegenerate,24.534453,14.660,0.334713
13,anfext2l_undegenerate,29.441344,17.305,0.340264
14,anfext2l_undegenerate,34.348234,19.665,0.349334
15,anfext2l_undegenerate,39.255125,22.260,0.352697
16,anfext2l_undegenerate,44.162015,24.965,0.353791
17,anfext2l_undegenerate,49.068906,27.160,0.361332
18,anfext2l_undegenerate,53.975797,28.325,0.381118
